# Pima Indian Model - Performance Evaluation
* Automatic Verification Dataset
* Manual Verification Dataset
* Manual k-Fold Cross Validation
* Using Scikit-Learn Hyper Parameters

### Automatic Verification Dataset

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers import Dense

seed = 7
numpy.random.seed(seed)

dataset = numpy.loadtxt("01-pima-indians-diabetes.csv", delimiter=',')

X = dataset[:,0:8]
Y = dataset[:,8]

model = Sequential()
model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,Y, validation_split=0.33, epochs=150, batch_size=10)

scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

### Manual Verification Dataset

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

seed = 7
numpy.random.seed(seed)

dataset = numpy.loadtxt("01-pima-indians-diabetes.csv", delimiter=',')

X = dataset[:,0:8]
Y = dataset[:,8]

X_train, X_test, Y_train, Y_test =  train_test_split(X,Y, test_size=0.33, random_state=seed)

model = Sequential()
model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=150, batch_size=10)

scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

### Manual k-Fold Cross Validation

In [ ]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

seed = 7
numpy.random.seed(seed)

def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    
dataset = numpy.loadtxt("01-pima-indians-diabetes.csv", delimiter=',')

X = dataset[:,0:8]
Y = dataset[:,8]

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10)

results = cross_val_score(model, X, Y, cv=kfold)
print("%.2f%%" % (results.mean()*100))


### Using Scikit-Learn Hyper Parameters

In [ ]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

seed = 7
numpy.random.seed(seed)

def create_model(optimizer='rmsprop', init='glorot_uniform'):
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    
dataset = numpy.loadtxt("01-pima-indians-diabetes.csv", delimiter=',')

X = dataset[:,0:8]
y = dataset[:,8]

model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10)

optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = numpy.array([50, 100, 150])
batches = numpy.array([5,10,20])
param_grid = dict(optimizer=optimizers, nb_epoch=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_result = grid.fit(X, y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_reult.cv_reults_['mean_test_scope']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))